In [16]:
!pip install transformers --upgrade
!pip install datasets
!pip install trl[peft] --upgrade
!pip install -U git+https://github.com/huggingface/trl
!pip install bitsandbytes loralib
!pip install wandb -U
!pip install hf_transfer
!pip install sentencepiece


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [17]:
%env HF_HUB_ENABLE_HF_TRANSFER=True
%env WANDB_PROJECT=LLM-Training-Course
%env WANDB_RUN_ID=DPO
%env WANDB_NOTEBOOK_NAME={__vsc_ipynb_file__}

env: HF_HUB_ENABLE_HF_TRANSFER=True
env: WANDB_PROJECT=LLM-Training-Course
env: WANDB_RUN_ID=DPO
env: WANDB_NOTEBOOK_NAME=/root/llm-training-course/04-dpo-orpo/01-train-with-dpo.ipynb


In [18]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [19]:
import sys
sys.path.append('/root/llm-training-course/')

In [20]:
from datasets import load_dataset

train_ds, eval_ds = load_dataset("mlabonne/orpo-dpo-mix-40k", split=["train[:20%]","train[20%:25%]"])

In [21]:
train_ds

Dataset({
    features: ['source', 'chosen', 'rejected', 'prompt', 'question'],
    num_rows: 8849
})

In [22]:
train_ds = train_ds.map(lambda x: { "messages": [{"role":"system", "content": x["prompt"] }] + x["chosen"] })
eval_ds = eval_ds.map(lambda x: { "messages": [{"role":"system", "content": x["prompt"] }] + x["chosen"] })

In [23]:
from transformers import AutoTokenizer

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/708a0609e640ac1edfb9020a7c934f51d34d6c79/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/708a0609e640ac1edfb9020a7c934f51d34d6c79/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/708a0609e640ac1edfb9020a7c934f51d34d6c79/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/708a0609e640ac1edfb9020a7c934f51d34d6c79/tokenizer_config.json


In [24]:
columns_to_remove = [c for c in train_ds.column_names if c not in ["chosen", "rejected", "prompt"]]
dpo_train_ds = train_ds.remove_columns(columns_to_remove)

columns_to_remove = [c for c in eval_ds.column_names if c not in ["chosen", "rejected", "prompt"]]
dpo_eval_ds = eval_ds.remove_columns(columns_to_remove)


In [25]:
dpo_train_ds = dpo_train_ds.map(lambda x: { "chosen": tokenizer.apply_chat_template(x["chosen"], tokenize=False),
                                            "rejected": tokenizer.apply_chat_template(x["rejected"], tokenize=False),
                                            "prompt": x["prompt"]})
dpo_eval_ds = dpo_eval_ds.map(lambda x: { "chosen": tokenizer.apply_chat_template(x["chosen"], tokenize=False),
                                            "rejected": tokenizer.apply_chat_template(x["rejected"], tokenize=False),
                                            "prompt": x["prompt"]})

In [26]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig
import torch


peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/708a0609e640ac1edfb9020a7c934f51d34d6c79/config.json
Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.3",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.42.4",
  "use_cache": true,
  "vocab_size": 32768
}

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snap

In [27]:
from helpers import get_gpu_status
get_gpu_status()

GPU = NVIDIA A40. Max memory = 44.352 GB.
27.889 GB of memory reserved.


In [28]:
from helpers import stream_responses_for_sample
from transformers import GenerationConfig

generation_config =  GenerationConfig(max_new_tokens=75)
sample_conversations = [
    [{"role": "user", "content": "What is the capital of France?"}],
    [{"role": "user", "content": "Write me a javascript function that check if string is palindrome."}],
    [{"role": "user", "content": "Given x^2=36-4 what is x?"}]
]
stream_responses_for_sample(model, tokenizer, sample_conversations,generation_config)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] What is the capital of France?[/INST] The 

capital of France is Paris. It is one of the most famous cities in the world, known for its rich history, art, culture, and landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also the political, economic, and cultural center of France.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Write me a javascript function that check if string is palindrome.[/INST] Here is a simple JavaScript function that checks if a string is a palindrome. This function will ignore spaces, punctuation, and convert the string to lowercase for comparison.

```javascript
function isPalindrome(str) {
   const re = /[\W_]/g;
   let cleanStr = str.toLowerCase().replace(


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Given x^2=36-4 what is x?[/INST] To find the value of x, you first need to rearrange the equation to isolate x. Here's how:

1. Add 4 to both sides of the equation:

x^2 + 4 = 36

2. Then, subtract 4 from both sides:

x^2 + 4 -


In [29]:
from trl import DPOTrainer, DPOConfig
import os
training_args = DPOConfig(
    output_dir=os.getenv("WANDB_RUN_ID") + "_DPO" ,
    report_to="wandb",
    num_train_epochs=1.0,
    do_train=True,
    do_eval=True,
    log_level="debug",
    gradient_checkpointing=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=4,
    lr_scheduler_type="constant",
    bf16=True,
    warmup_steps=150,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=0.1,
    max_grad_norm=.3,
    learning_rate=1e-6,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices


In [30]:
dpo_train_ds["chosen"][0]

'<s>[INST] The setting is an otherworldly, yet eerily familiar, metropolis known as "Zephyria." It\'s a city suspended in the ether, floating amidst nebulous clouds of cosmic dust. The architecture here is surreal and alien, with buildings that twist and spiral like strands of DNA, reaching towards the cosmos. Streets are paved with luminescent cobblestones, casting soft hues of blues and purples, reflecting off iridescent structures. Strange vegetation, vibrant and bioluminescent, thrives everywhere, creating a stark contrast against the deep indigo sky.\n\nNow, immerse me in this ethereal journey through Zephyria.[/INST] As you step onto the teleportation platform, there\'s a momentary sense of disorientation before your surroundings change abruptly. You find yourself standing on the outskirts of Zephyria, gazing at the sprawling metropolis that glows softly under the starlit canvas above. A gentle breeze, carrying hints of exotic fragrances from unknown flora, greets you. \n\nYou be

In [31]:

DPO_BETA=0.1

dpo_trainer = DPOTrainer(
    model,
    peft_config=peft_config,
    args=training_args,
    beta=DPO_BETA,
    train_dataset=dpo_train_ds,
    eval_dataset=dpo_eval_ds,
    tokenizer=tokenizer
)
dpo_trainer.train()
dpo_trainer.save_model()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:394: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:407: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:442: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You have loaded a model on multiple GPUs. `is_model_parall

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
56,0.689200,0.679826,-0.115637,-0.144267,0.589512,0.028629,-423.437836,-408.432068,-3.118704,-3.107814
112,0.682000,0.659986,-0.269820,-0.342939,0.649186,0.073119,-425.424622,-409.973938,-3.116685,-3.105672
168,0.673600,0.638503,-0.462601,-0.590397,0.667269,0.127796,-427.899139,-411.901703,-3.113685,-3.102568
224,0.657900,0.619901,-0.662428,-0.844780,0.679928,0.182351,-430.443024,-413.899963,-3.111378,-3.100127



***** Running Evaluation *****
  Num examples = 2212
  Batch size = 4

***** Running Evaluation *****
  Num examples = 2212
  Batch size = 4

***** Running Evaluation *****
  Num examples = 2212
  Batch size = 4

***** Running Evaluation *****
  Num examples = 2212
  Batch size = 4
Saving model checkpoint to DPO_DPO/checkpoint-276
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/708a0609e640ac1edfb9020a7c934f51d34d6c79/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": nu